# Classification and Model Selection

## Classifying Kickstarter Campaigns

Kickstarter is a crowdfunding platform with a community of more than 10 million people comprising of creative, tech enthusiasts who help in bringing new projects to life.

Until now, more than $3 billion dollars have been contributed by the members in fueling creative projects.
The projects can be literally anything – a device, a game, an app, a film, etc.

Kickstarter works on all or nothing basis: a campaign is launched with a certain amount they want to raise, if it doesn't meet its goal, the project owner gets nothing. For example: if a projects's goal is $\$5000$ and it receives $\$4999$ in funding, the project won't be a success.

If you have a project that you would like to post on Kickstarter now, can you predict whether it will be successfully funded or not? Looking into the dataset, what useful information can you extract from it, which variables are informative for your prediction and can you interpret the model?

The goal of this project is to build a classifier to predict whether a project will be successfully funded or not. 

**💡 You can use any algorithm of your choice.**

We will use `sklearn` and the usual data science libraries such as `pandas` and `numpy`.

KATE expects your code to define variables with specific names that correspond to certain things we are interested in.

KATE will run your notebook from top to bottom and check the latest value of those variables, so make sure you don't overwrite them.

* Remember to uncomment the line assigning the variable to your answer and don't change the variable or function names.
* Use copies of the original or previous DataFrames to make sure you do not overwrite them by mistake.

You will find instructions below about how to define each variable.

Once you're happy with your code, upload your notebook to KATE to check your feedback.

### Baseline Model

In this exercise, we are looking to outperform the performance of a simple `baseline` model. This `baseline` is a simple logistic regression with only two features: `goal_usd` (adjusted goal) and `usa` (whether the campaign happened in the US)

The code to build this `baseline` is shown below:

```Python
from sklearn.linear_model import LogisticRegression

# Conduct some custom processing on your training data
df["usa"] = df["country"] == "US"
df["goal_usd"] = df["goal"] * df["static_usd_rate"]

df = df[["goal_usd", "usa", "state"]]

# Conduct the same processing on your testing data
df_eval["usa"] = df_eval["country"] == "US"
df_eval["goal_usd"] = df_eval["goal"] * df_eval["static_usd_rate"]

df_eval = df_eval[["goal_usd", "usa", "state"]]

X = df.drop(["state"], axis=1)
y = df["state"]

X_eval = df_eval.drop(["state"], axis=1)

model = LogisticRegression()
model.fit(X, y)

y_pred = model.predict(X_eval)
```

### Our Model

To kick things off, let's import and use our favourite data processing library, `pandas`, to retrieve the data that we will use to build a machine learning model.

In this assignment, we are going to load in two datasets. The first, `df`, is going to contain all the data we will need to train and test a model. This will include the labels indicating whether or not the project was successfully funded. The second dataset, `df_eval`, is going to contain all the data that our model will be evaluated on by **KATE**. It does not include the labels indicating project success, so can be viewed as held-out test data. 

We will need to process `df_eval` in exactly the same way as `df`, then use our model trained on `df` to make predictions about `df_eval`. On submission, **KATE** will evaluate these predictions against their labels (which **KATE** has access to).


Run the cell below to load the raw data. Note that `pandas` is pretty smart and can read these ZipFiles into regular `DataFrames`:

In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



df = pd.read_csv("data/kickstarter.gz")
df_eval = pd.read_csv("data/kickstarter_eval.gz")

print(df.shape)
print(df_eval.shape)

(50000, 26)
(10000, 26)


We have also displayed the dimensions of our `df` and `df_eval`. Notice that the `df_eval` is only $10,000$ rows. As mentioned earlier, this will be our test set for submissions to **KATE**.

**This means that we cannot train on `df_eval`**


The aim of this practical is to:
  * Process `df` into an input dataframe `X` and a label dataframe `y`
  * Process `df_eval` into an input dataframe `X_eval` (in the same way as we processed `df` into `X`)
  * Train a classification model of our choice on `X` and `y`
  * Submit our code to KATE, where our model will be evaluated on `X_eval` and `y_eval`

Let's kick things off by checking out our `df`. Note that the `state` column contains our success labels:

In [2]:
df.head()

,id,photo,name,blurb,goal,slug,disable_communication,country,currency,currency_symbol,...,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions,state
0,805910621,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",DOCUMENTARY FILM titled FROM RAGS TO SPIRITUAL...,A MOVIE ABOUT THE WILLINGNESS TO BREAK FREE FR...,125000.0,movie-made-from-book-titled-from-rags-to-spiri...,False,US,USD,$,...,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,0.0
1,1279627995,"{""small"":""https://ksr-ugc.imgix.net/assets/011...","American Politics, Policy, Power and Profit",Everything you should know about really big go...,9800.0,american-politics-policy-power-and-profit,False,US,USD,$,...,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,0.0
2,1306016155,"{""small"":""https://ksr-ugc.imgix.net/assets/013...","Drew Jacobs Official ""Kiss Me"" Music Video","Be a part of the new ""Kiss Me"" Official Music ...",2500.0,drew-jacobs-official-kiss-me-music-video,False,US,USD,$,...,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,1.0
3,658851276,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Still Loved,When their dreams are shattered by the loss of...,10000.0,still-loved,False,GB,GBP,Â£,...,"{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,1.0
4,1971770539,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Nine Blackmon's HATER Film Project,HATER is a mock rock doc about why the Rucker ...,5500.0,nine-blackmons-hater-film-project,False,US,USD,$,...,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,0.0


In addition to using the `.head()` function, let's also retrieve some more information about our data:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      50000 non-null  int64  
 1   photo                   50000 non-null  object 
 2   name                    49998 non-null  object 
 3   blurb                   49998 non-null  object 
 4   goal                    50000 non-null  float64
 5   slug                    50000 non-null  object 
 6   disable_communication   50000 non-null  bool   
 7   country                 50000 non-null  object 
 8   currency                50000 non-null  object 
 9   currency_symbol         50000 non-null  object 
 10  currency_trailing_code  50000 non-null  bool   
 11  deadline                50000 non-null  int64  
 12  created_at              50000 non-null  int64  
 13  launched_at             50000 non-null  int64  
 14  static_usd_rate         50000 non-null

From the above, we can see that there are $50,000$ projects in `df` and, apart from a handful of columns, most of our data is not null - what a relief!

In [4]:
df_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      10000 non-null  int64  
 1   photo                   10000 non-null  object 
 2   name                    10000 non-null  object 
 3   blurb                   10000 non-null  object 
 4   goal                    10000 non-null  float64
 5   slug                    10000 non-null  object 
 6   disable_communication   10000 non-null  bool   
 7   country                 10000 non-null  object 
 8   currency                10000 non-null  object 
 9   currency_symbol         10000 non-null  object 
 10  currency_trailing_code  10000 non-null  bool   
 11  deadline                10000 non-null  int64  
 12  created_at              10000 non-null  int64  
 13  launched_at             10000 non-null  int64  
 14  static_usd_rate         10000 non-null 

Unlike `df`, `df_eval` contains only $10,000$ projects. Also note that the target variable, `state` is null for all entries - as mentioned earlier, this is stored on **KATE** for evaluating our model when we submit our code.

In [5]:
print("df labels:      ", df.state.unique())
print("df_eval labels: ", df_eval.state.unique())

df labels:       [0. 1.]
df_eval labels:  [nan]


**Notes on the dataset**:
* The target `state` corresponds to a binary outcome: `0` for failed, `1` for successful. 
* The variables `'deadline'`, `'created_at'`, `'launched_at'` are stored in Unix time format.

## Part 1: Preprocessing the data

Although our data is relatively clean, it is not yet in a state where we can train a model. For instance, both `df` and `df_eval` contains columns used for training (features) as well as the target column although this is, of course, null for `df_eval`.

What we have to do now is preprocess our data. Specifically, we need to:
 - Build a training set: `X` and `y`
 - Build an evaluation set: `X_eval`
 
 <br>
 
Let's start by extracting the `state` column from `df` into a variable called `y`:
 - Create a new variable `y` from `df["state"]`
 - Drop the `state` column from `df` and `df_eval`

In [6]:
y = df["state"]

df.drop("state", axis=1, inplace=True)
df_eval.drop("state", axis=1, inplace=True)

### 1.1 Remove redundant columns

After removing the target variable `y` from our input data, we can start processing.

Remove all the columns that **you** think are not salient for this classification task. For instance, `id`, `photo`, `slug`, and `disable_communication` are some features which are not likely to be relevant. The choice of which features to retain, however, is yours to make. Remember to remove the same columns from `df` as `df_eval`.

You can use the `.drop()` function from `pandas` to remove columns (remember to specify `axis=1`)

In [7]:
pd.set_option("display.max_columns", 500)

In [8]:
df.head(1)

,id,photo,name,blurb,goal,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,created_at,launched_at,static_usd_rate,creator,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions
0,805910621,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",DOCUMENTARY FILM titled FROM RAGS TO SPIRITUAL...,A MOVIE ABOUT THE WILLINGNESS TO BREAK FREE FR...,125000.0,movie-made-from-book-titled-from-rags-to-spiri...,False,US,USD,$,True,1447162860,1444518329,1444673815,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [9]:
def get_cats(data):
    json_column = data["category"].apply(json.loads).values
    cats = [json_column[i]["slug"] for i in range(len(json_column))]

    data["ex_category"] = cats
    
    return data

df = get_cats(df)
df_eval = get_cats(df_eval)

In [10]:
df.head()

,id,photo,name,blurb,goal,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,created_at,launched_at,static_usd_rate,creator,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions,ex_category
0,805910621,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",DOCUMENTARY FILM titled FROM RAGS TO SPIRITUAL...,A MOVIE ABOUT THE WILLINGNESS TO BREAK FREE FR...,125000.0,movie-made-from-book-titled-from-rags-to-spiri...,False,US,USD,$,True,1447162860,1444518329,1444673815,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,film & video/movie theaters
1,1279627995,"{""small"":""https://ksr-ugc.imgix.net/assets/011...","American Politics, Policy, Power and Profit",Everything you should know about really big go...,9800.0,american-politics-policy-power-and-profit,False,US,USD,$,True,1351709344,1348156038,1349117344,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,publishing/nonfiction
2,1306016155,"{""small"":""https://ksr-ugc.imgix.net/assets/013...","Drew Jacobs Official ""Kiss Me"" Music Video","Be a part of the new ""Kiss Me"" Official Music ...",2500.0,drew-jacobs-official-kiss-me-music-video,False,US,USD,$,True,1475174031,1473271187,1473359631,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,music/country & folk
3,658851276,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Still Loved,When their dreams are shattered by the loss of...,10000.0,still-loved,False,GB,GBP,Â£,False,1400972400,1395937256,1397218790,1.680079,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,film & video/documentary
4,1971770539,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Nine Blackmon's HATER Film Project,HATER is a mock rock doc about why the Rucker ...,5500.0,nine-blackmons-hater-film-project,False,US,USD,$,True,1425963600,1422742820,1423321493,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN,film & video/narrative film


In [11]:
# Your code here:
columns_to_drop = ["id", "photo", "slug", "disable_communication", "friends", "is_starred", "permissions", "source_url", "creator", "location", "profile", "urls", "category", "currency_symbol", "is_backing"]

df.drop(columns=columns_to_drop, inplace=True)
df_eval.drop(columns=columns_to_drop, inplace=True)

### 1.2 Fill null values

Looking at the output of `df.info()` above, we can see that some of the columns which we might be interested in as features contain some null values. Null values are, in general, a problem for machine learning models and can cause your code to break. How you choose to deal with them, however, will depend in large part on how you intend to process your data. For instance, if your input data consists of strings that you wish to generate a word count feature from, you can just fill in the null values with empty strings (`""`).

Thankfully, `pandas` has a helpful function for dealing with null values: the `.fillna()` function. Remember to do the same to `df` as `df_eval`:

In [12]:
df.isna().any()

name                       True
blurb                      True
goal                      False
country                   False
currency                  False
currency_trailing_code    False
deadline                  False
created_at                False
launched_at               False
static_usd_rate           False
ex_category               False
dtype: bool

In [13]:
# Your code here:


### 1.3 Additional Processing

In the previous two exercises we have covered the most basic steps in preprocessing: dropping redundant columns and working with null values. However, there is *so* much more that we can do to extract useful information from our data. 

For instance, the `blurb` column contains unique strings and so, in its current form, isn't a particularly useful feature. Instead, we could create a new feature representing the length of the `blurb`, or the number of words in the `blurb`. 

Other string-type columns, such as `country`, contain categorical data. As there are a lot of countries represented, we might want to aggregate these into regions (e.g. `Europe`, `Asia`, ...). We can then convert this categorical data into a one-hot encoding using `sklearn`.

What we are describing here is what's known as feature engineering and is an art and a science in its own right. 

Let's start this processing by importing some libraries and functions that can help us create features. Notice that we import the `StandardScaler` from `sklearn`. We can use this function on our numerical data to normalise it, which is an important step in training a machine learning model.

**💡 In the following cell, you can use feature engineering to create features that you think might be useful.**

<br>

You may want to put all your processing within a function (such as `processing()`) or may want to do it just as plain Python code. It's entirely up to you!

However, once you have processed `df` and `df_eval`, you must assign them to input variables `X` and `X_eval`.

In [14]:
df.head()

,name,blurb,goal,country,currency,currency_trailing_code,deadline,created_at,launched_at,static_usd_rate,ex_category
0,DOCUMENTARY FILM titled FROM RAGS TO SPIRITUAL...,A MOVIE ABOUT THE WILLINGNESS TO BREAK FREE FR...,125000.0,US,USD,True,1447162860,1444518329,1444673815,1.000000,film & video/movie theaters
1,"American Politics, Policy, Power and Profit",Everything you should know about really big go...,9800.0,US,USD,True,1351709344,1348156038,1349117344,1.000000,publishing/nonfiction
2,"Drew Jacobs Official ""Kiss Me"" Music Video","Be a part of the new ""Kiss Me"" Official Music ...",2500.0,US,USD,True,1475174031,1473271187,1473359631,1.000000,music/country & folk
3,Still Loved,When their dreams are shattered by the loss of...,10000.0,GB,GBP,False,1400972400,1395937256,1397218790,1.680079,film & video/documentary
4,Nine Blackmon's HATER Film Project,HATER is a mock rock doc about why the Rucker ...,5500.0,US,USD,True,1425963600,1422742820,1423321493,1.000000,film & video/narrative film


In [15]:
country_regions = {
    "US": "NA",
    "GB": "Europe",
    "CA": "NA",
    "DK": "Europe",
    "AU": "APAC",
    "IT": "Europe",
    "NZ": "APAC", 
    "DE": "Europe",
    "ES": "Europe",
    "AT": "Europe",
    "IE": "Europe",
    "NL": "Europe",
    "SE": "Europe",
    "MX": "LA",
    "FR": "Europe",
    "BE": "Europe",
    "NO": "Europe",
    "CH": "APAC",
    "SG": "APAC",
    "HK": "APAC",
    "LU": "Europe",
}

In [16]:
import json
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

# Your code here:
def processing(data):
    # drop ex_category
    data = data.drop(["ex_category"], axis=1)
    
    # feature engineer blurb/name
    data["len_blurb"] = data["blurb"].str.len()
    data["num_words"] = data["blurb"].str.split().str.len() 
    
    data["len_name"] = data["name"].str.len()
    data["num_name"] = data["name"].str.split().str.len() 
    
    data = data.drop(["blurb", "name"], axis=1)

    # add regions
    data["region"] = data["country"].map(country_regions)
    dummies = pd.get_dummies(data["region"], drop_first=False).astype(int)
    data = pd.concat([data, dummies], axis=1)
    data = data.drop(["country", "region"], axis=1)

    # encode currencies
    dummies = pd.get_dummies(data["currency"], drop_first=False).astype(int)
    data = pd.concat([data, dummies], axis=1)
    data = data.drop(["currency"], axis=1)
    
    # unix to timestamp
    create_date = pd.to_datetime(data["created_at"], unit="s")
    deadline = pd.to_datetime(data["deadline"], unit="s")
    launched_date = pd.to_datetime(data["launched_at"], unit="s")
    data["days_until_deadline"] = (deadline - create_date).dt.days
    data["create_to_launch"] = (launched_date - create_date).dt.days
    data = data.drop(["created_at", "launched_at", "deadline"], axis=1)
    
    
    # bin_goal
        # data["goal"] = data["goal"] * df["static_usd_rate"]
        # conds = [
        #     (data["goal"] <= 200),
        #     ((data["goal"] > 200) & (data["goal"] <= 4000)),
        #     ((data["goal"] > 4000) & (data["goal"] <= 10_000)),
        #     ((data["goal"] > 10_000) & (data["goal"] <= 50_000)),
        #     (data["goal"] > 50_000)
        # ]
        
        # choice = [1, 2, 3, 4, 5,]
        # data["goal_bin"] = np.select(conds, choice, data["goal"])
        # data = data.drop(["goal"], axis=1)
    
    
    data = data.drop(["static_usd_rate", "currency_trailing_code"], axis=1)
        
    # scale data
    scaler = StandardScaler()
    # df["goal"] = scaler.fit_transform(df["goal"].values.reshape(-1,1))
    
    return data.fillna(0)

X = processing(df)
X_eval = processing(df_eval)

print(X.shape, X_eval.shape)

(50000, 24) (10000, 24)


In [17]:
X.head()

,goal,len_blurb,num_words,len_name,num_name,APAC,Europe,LA,NA,AUD,CAD,CHF,DKK,EUR,GBP,HKD,MXN,NOK,NZD,SEK,SGD,USD,days_until_deadline,create_to_launch
0,125000.0,134.0,24.0,53.0,8.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,30,1
1,9800.0,131.0,19.0,43.0,6.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,41,11
2,2500.0,52.0,11.0,42.0,7.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,22,1
3,10000.0,120.0,22.0,11.0,2.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,58,14
4,5500.0,125.0,23.0,34.0,5.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,37,6


## Part 2: Training the model

Now that we have separated our data into train and evaluation data, we can start training models and evaluating their performance. At this point, you are welcome to explore any model architecture, so long as it is a **classification** model.

Check out the `sklearn` [documentation](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) for a selection of possible models and implementation examples.

Note that most `sklearn` models have the same interface. Once imported you can create an instantiation of your model (specifying custom settings as you see fit), and assign it to a variable. For instance:

``` Python
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
```

Once you have created your `model` variable, you can call `.fit()` and pass `X` and `y` as arguments.

**💡 For KATE to work, your model must be assigned to a variable called `model`**

**NOTE**: Since with this project your model will be trained directly on KATE, it is limited to models that can be trained under 1min. You will receive a `TimeoutError` if your model takes too long.


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb
import scipy.stats as stats

In [38]:
stats.randint(50, 400).

{'t': <scipy.stats._distn_infrastructure.rv_discrete_frozen at 0x26916136bb0>}

In [40]:
import scipy.stats as stats

# Define the hyperparameter distributions
param_dist = {
    "max_depth": stats.randint(3, 20),
    "min_child_weight": stats.randint(3, 20),
    "learning_rate": stats.uniform(0.001, 0.1),
    "subsample": stats.uniform(0.5, 0.5),
    "n_estimators": stats.randint(50, 400),
    "subsample": np.arange(0.05, 1.0, 0.01),
    "colsample_bytree": np.arange(0.05, 1.0, 0.01),
    "colsample_bylevel": np.arange(0.05, 1.0, 0.01),
    "scale_pos_weight": np.arange(1, 32, 5)
}

# Create the XGBoost model object
xgb_model = xgb.XGBClassifier()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring="accuracy")

random_search.fit(X, y)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
       0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37,
       0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
       0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59,
       0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 ,
       0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81,
       0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92,
       0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])},
                   scoring='accuracy')

In [41]:
random_search.best_params_

{'colsample_bylevel': 0.9600000000000002,
 'colsample_bytree': 0.8800000000000002,
 'learning_rate': 0.09288047809173726,
 'max_depth': 4,
 'min_child_weight': 15,
 'n_estimators': 337,
 'scale_pos_weight': 1,
 'subsample': 0.08000000000000002}

In [43]:
best_params = {
    "colsample_bylevel": 0.8600000000000002,
    "colsample_bytree": 0.8200000000000002,
    "learning_rate": 0.03572523708482918,
    "max_depth": 7,
    "min_child_weight": 9,
    "n_estimators": 245,
    "subsample": 0.9700000000000002,
}

In [45]:
model = xgb.XGBClassifier(**random_search.best_params_)

model = model.fit(X, y)

Once trained, we can use the `.score()` function to evaluate our model's performance on the train set. Remember to pass `X` and `y` as arguments.

In [46]:
# Your code here:
model.score(X, y)

0.65008

## Part 3: Making predictions

Now that our model is trained, we can use the `.predict()` function to make predictions for the rows in our data where `y` is not known.
 - Call `.predict()` on the `model` variable, and pass `X_eval`
 - Assign the output of `.predict()` to a variable called `y_pred`

In [ ]:
# Your code here:
y_pred = model.predict(X_eval)

Note that in the previous exercise, we used the `.score()` function to evaluate our model on the training data. However, we do not have the ground truth for our `X_eval` data points - to see how well the model performs on the test set, you will have to submit it to **KATE**!